# Microsoft Power BI Cmdlets for Windows PowerShell and PowerShell Core
The following cmdlets are available on PowerShell Gallery and can be installed in an elevated PowerShell session:
- [Data Gateway](https://docs.microsoft.com/en-us/powershell/module/datagateway/?view=datagateway-ps)

In [ ]:
$modules = @("DataGateway")

foreach ( $m in $modules ) 
{
    if (Get-Module -ListAvailable -Name $m) {
        write-host "Module $m is already imported."
    } 
    else {
        Install-Module -Name $m -Force -Scope CurrentUser
        Import-Module $m
    }
}

# Available Cmdlets
Returns the available cmdlets from either the MicrosoftPowerBIMGMT or DataGateway module.

In [ ]:
Get-Help *DataGateway* | Sort-Object Name | Format-Table -AutoSize

# Connect to Data Gateway

Connect to the Data Gateway service with either a user or service principal account (application secret or certificate).

For user accounts, an Azure Active Directory (AAD) First-Party application is leveraged for authentication.

**For Government:** Connect-DataGatewayServiceAccount -Environment USGov

Learn More: [Examples](https://docs.microsoft.com/en-us/powershell/module/datagateway.profile/connect-datagatewayserviceaccount?view=datagateway-ps#examples)

In [ ]:
Connect-DataGatewayServiceAccount

# Scope

Indicates the scope of the call.
- -Scope Individual
    - Returns only gateways assigned to the caller.
- -Scope Organization
    - Returns all gateways within a tenant (You must be an O365 tenant admin, Power Platform admin or a Power BI admin).

In [ ]:
$scopeLevel = "Individual"

# Return a list of Data Gateways

Get gateway clusters for which the current user is an administrator.

In [ ]:
$gateways = Get-DataGatewayCluster -Scope $scopeLevel

Write-Host "Total Number of Gateways: $($gateways.Count)"

$gateways

# Data Gateway Statuses

In [ ]:
$dataGateways = Get-DataGatewayCluster -Scope $scopeLevel

$gateways = @()

ForEach ($dataGateway in $dataGateways) {
    
    $gateways += Get-DataGatewayClusterStatus -GatewayClusterId $dataGateway.Id
                | Select *, @{Name="GatewayId"; Expression={ $dataGateway.Id } }

}

$gateways

# Gateway Upgrades

Get a list of gateways where a new version is available.

In [ ]:
$dataGateways = Get-DataGatewayCluster -Scope $scopeLevel

$gateways = @()

ForEach ($dataGateway in $dataGateways) {
    
    $gateways += Get-DataGatewayClusterStatus -GatewayClusterId $dataGateway.Id 
                | Where-Object {$_.GatewayUpgradeState -eq "NewVersionAvailable"} 
                | Select *, @{Name="GatewayId"; Expression={ $dataGateway.Id } } 
                
}

Write-Host "Total Gateways with a new version available: $($gateways.Count)"

$gateways

# Return Member Data Gateways

Get details of each member gateway.

In [ ]:
$dataGateways = Get-DataGatewayCluster -Scope $scopeLevel

$gateways = @()

ForEach ($dataGateway in $dataGateways) {
    
    $gateways += Get-DataGatewayCluster -GatewayClusterId $dataGateway.Id
                | Select -ExpandProperty MemberGateways

}

$gateways | Sort-Object -Property ClusterId

# Return Data Gateway Permissions

Get a list of users with permissions to administer or use the gateway cluster

In [ ]:
$dataGateways = Get-DataGatewayCluster -Scope $scopeLevel

$gateways = @()

ForEach ($dataGateway in $dataGateways) {
    
    $gateways += Get-DataGatewayCluster -GatewayClusterId $dataGateway.Id
                | Select -ExpandProperty Permissions

}

$gateways | Sort-Object -Property ClusterId

# Data Gateway Details

Gets all gateway regions along with an identification of the default Power BI region for your tenant.

In [ ]:
Get-DataGatewayRegion

# Data Gateway Tenant Policy

Returns Tenant Policy Information

In [ ]:
Get-DataGatewayTenantPolicy

# Enable on-premises and cloud data sources on cluster

You can use the on-premises data gateway to merge or append on-premises and cloud data sources in the same query. This solution is helpful when you want to combine data from multiple sources without having to use separate queries.

<b>Note:</b> The following script enables for all data gateways.

Learn More: [Merge or append on-premises and cloud data sources](https://docs.microsoft.com/en-us/power-bi/connect-data/service-gateway-mashup-on-premises-cloud)

In [ ]:
$dataGateways = Get-DataGatewayCluster -Scope $scopeLevel

ForEach ($dataGateway in $dataGateways) {
  
    Set-DataGatewayCluster -GatewayClusterId $dataGateway.Id -AllowCloudDatasourceRefresh $true
    Write-Host "Cloud Data Source Refresh Now Enabled: $($dataGateway.Id)" 

}